In [ ]:
import marqo
import sys
import pathlib

In [ ]:
import numpy as np

In [ ]:
sys.path.insert(0, "src")
import data_functions
import embedding_functions
import cluster_functions
import graph_functions
import util_functions

In [ ]:
base_name = "grascco_test"
path_name = lambda x: f"./tmp/{base_name}/{base_name}_{x}.pickle"

In [ ]:
grascco_embedding = embedding_functions.SentenceEmbeddingsFactory.load(
    pathlib.Path(path_name("data")),
    pathlib.Path(path_name("embedding"))
)

In [ ]:
grascco_embedding.data_processing_obj.data_chunk_sets[1]

In [ ]:
def create_index_document(sentence_embedding_obj: 'embedding_functions.SentenceEmbeddingsFactory.SentenceEmbeddings'):
    for i, embedding in enumerate(sentence_embedding_obj.sentence_embeddings):
        yield {
            "id": i,
            "phrase": sentence_embedding_obj.data_processing_obj.data_chunk_sets[i]['text'],
            "phrase_vector": {
                "vector": [float(f) for f in embedding]
            }
        }

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

In [ ]:
index_name = "test_index"
settings = {
    "treatUrlsAndPointersAsImages": False,
    "model": "no_model",
    "modelProperties": {
        "dimensions": 1024,
        "type": "no_model",
    },
}
try:
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )
except:
    mq.delete_index(index_name)
    mq.create_index(
        index_name=index_name,
        settings_dict=settings,
    )

In [ ]:
mq.index("test_index").add_documents(
    list(create_index_document(grascco_embedding)),
    client_batch_size=128,
    tensor_fields=["phrase_vector"],
    mappings={
        "phrase_vector": {
            "type": "custom_vector"
        }
    },
)

In [41]:
mq.index("test_index").

TypeError: Index.get_documents() missing 1 required positional argument: 'document_ids'